### Parse de pronoturf. Clasificación y estadísticas 2020

Esto es un parse de https://www.pronoturf.es/estadisticas

Se guarda como: data_pronoturf

456 caballos

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [40]:
import numpy as np

In [2]:
url = 'https://www.pronoturf.es/estadisticas'

page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
tbl = soup.find("table",{"id":"table_detalle_xs"})

data_frame = pd.read_html(str(tbl))[0]

In [4]:
data_frame

,#,NOMBRE,PADRE,PREPARADOR,PROPIETARIO,VALOR,C.C,C.G,C.COL,GANANCIA
0,1,NAVIA (IRE),LOPE DE VEGA,G. Arizkorreta,Yeguada Rocio,42.5,5,2,3,"40,500 €"
1,2,ABU,RODERIC O'CONNOR,J.M. Osorio,Santa Bárbara,42.0,5,3,2,"34,400 €"
2,3,PRAVIA,DAWN APPROACH,G. Arizkorreta,Yeguada Rocio,41.0,4,2,2,"32,600 €"
3,4,NORAY (FR),NAAQOOS,E. Leon,Marques de Miraflores,44.0,3,2,1,"32,300 €"
4,5,CABUERNIGA (GB),TEOFILO,P. Olave,Bering,34.0,4,2,2,"29,600 €"
...,...,...,...,...,...,...,...,...,...,...
451,452,AYAMONTE (FR),WHIPPER,J. Oliveira,Ecurie JNK,23.5,1,0,1,200 €
452,453,SANDY DUKE (FR),DUKE OF MARMALADE,J. Oliveira,Maia Lidador,21.5,1,0,1,200 €
453,454,NIAGARA,VIVA A FRANCA,J. Horcajada,Yeguada Cortiñal,21.5,1,0,1,200 €
454,455,POPOTA (GB),MIZZEN MAST,M. Alvarez,Celso Mendez,21.0,1,0,1,200 €


C.C = Carreras corridas

C.G = Carreras ganadas

C.Col = Carreras colocado

In [5]:
data_frame.to_csv(r'C:\Users\norio\Documents\Hipodromo\data_pronoturf.csv', sep = ",")

### Parse de gaceta hipodromo. Clasificación y estadísticas 2020

Esto es un parse de https://www.gacetahipodromo.com/gh_stats/horses

Se guarda como: data_gaceta

100 caballos

Sobre la anterior, añade que te dice resultados de 1º, 2º y demás

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.gacetahipodromo.com/gh_stats/horses'

page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

In [23]:
tbl = soup.findAll('table')
#tbl = soup.find("table",{"id":"datatable"})
print(tbl)

[]


No lo he conseguido.

Lo he hecho copiando directamente y pegando en un txt.
Replace de los tabuladores por comas y se tiene ya como csv

### Parse de hipódromo zarzuela. Todos los resultados

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
ID_tabla = "datatable"
url_root = "https://www.hipodromodelazarzuela.es"

In [3]:
# Deprecated porque no quería hacer dos llamadas al url
def obtener_df_deprecated(url, ID):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    tbl = soup.find("table",{"id":ID})
    data_frame = pd.read_html(str(tbl))[0]
    return data_frame

In [4]:
def obtener_df(soup, ID):
    tbl = soup.find("table",{"id":ID})
    data_frame = pd.read_html(str(tbl))[0]
    return data_frame

In [86]:
def obtener_datos_carrera(soup):
    tbl = soup.find("table",{"class":"table table-bordered table-condensed display"})
    data_frame = pd.read_html(str(tbl))[0]
    
    # No cogemos las columnas de día y hora
    cols = [2,3,4,5]
    interesting_data = data_frame[data_frame.columns[cols]]
    dia_carrera = data_frame[data_frame.columns[0]]
    hora_carrera = data_frame[data_frame.columns[1]]
    
    # Se convierten los datos a array
    interesting_data_array = interesting_data.iloc[0].values
    
    # Vector donde se almacenarán los datos de la carrera
    datos_carrera_sucios = []
    # Se recorre el vector con los datos de la carrera
    for i in range(0,len(interesting_data_array)):
    
        # Se obtiene el dato individual
        dato_sucio = interesting_data_array[i]
    
        # Se obtiene la información a la derecha del ":"
        info = dato_sucio.split(":")[1]
        
        # Se recoge toda esta información en el vector de datos de la carrera
        datos_carrera_sucios.append(info)
    
    
    # Ahora se limpia el vector
    
    # Terreno
    terreno= datos_carrera_sucios[0].replace(" ", "")

    # Distancia
    distancia_string = datos_carrera_sucios[1][1:-2]
    distancia_string_corrected = distancia_string.replace(".", "")
    distancia = int(distancia_string_corrected)

    # Tipo
    tipo = datos_carrera_sucios[2].replace(" ", "")

    # Tiempo
    # Se obtiene el tiempo separado por ":"
    tiempo_aux_1 = datos_carrera_sucios[3].replace("'", ":")
    tiempo_aux_2 = tiempo_aux_1.replace("\"", ":")
    tiempo_string = tiempo_aux_2.replace(" ", "")

    # Se obtienen el tiempo total en segundos
    minutos = 0;
    segundos = 0;
    try:
        # A veces esto está metido de una manera u otra
        minutos = tiempo_string.split(":")[0]
        segundos = tiempo_string.split(":")[1]
    except IndexError:
        minutos = 0;
        segundos = 0;
        print("Error en carrera" , str(dia_carrera) , str(hora_carrera)), 
        
    
    tiempo = int(minutos)*60 + int(segundos)

    
    datos_carrera = [terreno, distancia, tipo, tiempo]
    
    
    return datos_carrera

In [78]:
def obtener_df_carreras_dia(links_carreras_dia):
    # Obtenemos todos los dataframes
    array_data_frames = []
    for link in links_carreras_dia:
        cod_carrera = link.split("/")[-2] # Se obtiene el código de la carrera que está al final de link
        cod_dia = link.split("/")[-1] # Se obtiene el código del dia que está al final de link
        
        # Se obtiene el soup en el link del día
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        
        # Se obtiene la tabla de datos con las carreras del día
        df_carrera = obtener_df(soup, ID_tabla)
        
        
        # Se obtiene la info del día
        datos_carrera = obtener_datos_carrera(soup)
        
        
        # Se añade la info del día a cada una de las entradas de la carrera
        df_carrera['Terreno'] = datos_carrera[0]
        df_carrera['Distancia'] = datos_carrera[1]
        df_carrera['Tipo'] = datos_carrera[2]
        df_carrera['Tiempo'] = datos_carrera[3]
        
        
        # Se añaden datos particulares para saber el día y la carrera
        df_carrera['CodCarrera'] = cod_carrera
        df_carrera['CodDia'] = cod_dia
        array_data_frames.append(df_carrera)
    
    # Se juntan en un solo dataframe
    data_frame_day = array_data_frames[0]
    for i in range(1, len(array_data_frames)):
        data_frame_day = pd.concat([data_frame_day, array_data_frames[i]])
    
    return data_frame_day

In [7]:
# url_jornada es: /carreras/jornada/20200614
def obtener_links_carreras_dia(url_jornada):
    url = url_root + url_jornada
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Se obtienen los links a las carreras del día
    table = soup.find('table', {"id": "datatable-jornada"})
    all_hrefs = table.findAll('a', )
    
    numero_links = len(all_hrefs)

    # Se obtiene el numero de carreras
    numero_carreras = int(numero_links / 3)

    # Se obtienen únicamente los hrefs que son de una carrera
    href_carreras = []
    for i in range(0,numero_carreras):
        href_carreras.append(all_hrefs[i*3])

    # Se obtienen los link de las carreras
    link_carreras = []
    for link_carrera in href_carreras:
        link_carreras.append(url_root+link_carrera.attrs["href"])
    
    return link_carreras

**Este es un ejemplo de una carrera de un día**

In [8]:
un_dia = obtener_df_deprecated('https://www.hipodromodelazarzuela.es/carreras/carrera/0000046213/20201002',"datatable" )
un_dia

,Puesto,Caballo,Peso,Edad,Dist.,Mant.,Divid.,Propietario,Preparador,Jinete
0,1,SIR ROQUE (FR) (a),5600-5450,4,NaN,2,190,YEGUADA AGF,G.ARIZKORRETA,E. ARGUINZONES
1,2,PELLIQUERO (FR),6100,3,1 C 1/2,1,370,CIELO DE MADRID,J.A.RODRIGUEZ,J.L. BORREGO
2,3,HEADHUNTER (FR),5400-5000,3,6 C 3/4,4,290,PALCO 7,J.L.MAROTO,A.MARTINEZ
3,4,MARAVEDI (IRE),5400-5450,3,20 C,5,1470,ODISEA,A.CARRASCO,R. RAMOS
4,5,GACEFAR (GB),5450,4,27 C 1/2,3,2140,GANADERIA JUMAAN,"FUSTE, J.",F.MARTINEZ


**Para obtener todas las carreras de un día**

In [11]:
links_un_dia = obtener_links_carreras_dia("/carreras/jornada/20190414")
df = obtener_df_carreras_dia(links_un_dia)
df.head()


,Puesto,Caballo,Peso,Edad,Dist.,Mant.,Divid.,Propietario,Preparador,Jinete,Terreno,Distancia,Tipo,Tiempo,CodCarrera,CodDia
0,1,ALGAIDA (FR),6250,4,NaN,7,740,FUENTE EL SOL,G.ARIZKORRETA,"SR. GREDILLA,P.",H-Bueno,1800,Liso,121.3,44080,20190414
1,2,ANGLIRU (IRE),6500,4,3 C 1/2,3,240,YEGUADA ROCIO,G.ARIZKORRETA,"SR. PINEDA, G.",H-Bueno,1800,Liso,121.3,44080,20190414
2,3,BILLY BUZZ (FR),6400-6300,4,CABEZA,4,410,SALVADOR MARQUEZ,J.L.MAROTO,"SR. MAROTO, R.",H-Bueno,1800,Liso,121.3,44080,20190414
3,4,BARRY LYNDON (FR),6800,5,3/4 C,1,320,RIO CUBAS,O.ANAYA,"SR.SARABIA,D.A.",H-Bueno,1800,Liso,121.3,44080,20190414
4,5,QUICK ARTIST (IRE),6550,5,5 C,2,2600,EACHWAY,A.SANCHEZ C.,"SRTA.BROTHERTON,...",H-Bueno,1800,Liso,121.3,44080,20190414


In [236]:
df.shape

(35, 16)

**Para obtener una lista de todos los días que hay carrera**

Esto no se ha conseguido

In [8]:
url = "https://www.hipodromodelazarzuela.es/carreras#calendario-resultados"

In [9]:
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [21]:
#a = soup.find_all('a')
calendarios = soup.find_all('div', {"class": "tab-content"})

In [24]:
calendario_2020 = soup.find_all('div', {"id": "calendario-carreras-2020"})
calendario_2020

[<div id="calendario-carreras-2020"></div>]

**Obteniendo a mano una tabla de varios días de carreras**

Se hace un vector con los códigos de día de cada uno de los años

In [67]:
dias_2020 = ["20200301","20200308","20200614", "20200621", "20200628", "20200705", "20200709", "20200712", "20200716", "20200723", "20200730", "20200806", "20200813", "20200818", "20200828", "20200910", "20200913", "20200917", "20200920", "20200924", "20200927", "20201002", "20201004", "20201009", "20201011", "20201016"]

In [68]:
dias_2019 = ["20190303","20190310","20190317","20190324","20190331","20190407","20190414","20190421","20190428","20190505","20190512","20190519","20190526","20190602","20190609","20190615","20190622","20190629","20190704","20190718","20190725","20190808","20190915","20190922","20190929","20191006","20191013","20191020","20191027","20191103","20191110","20191117","20191124","20191201","20191226"]
# "20190303" ommited for format in Time of race is ' '  instead of " for decimals of seconds
# 20190711" "20190801" ommited for not having photo finish

In [69]:
dias = dias_2019 + dias_2020

Iterando el vector anterior, se crea un vector de links

In [70]:
links = []
for i in dias:
    link = "/carreras/jornada/" + i;
    links.append(link)
    
    

Se hace un loop obteniendo los links de cada día, obteniendo las bases de datos de cada uno y concatenando las distintas tablas

In [71]:
df_races = pd.DataFrame()

Para cada link, se obtienen los datos y se va concatenando la base de datos hasta obtener una de los dos años

In [87]:
for dia in links:
    print(dia)
    links_un_dia = obtener_links_carreras_dia(dia)
    df_aux = obtener_df_carreras_dia(links_un_dia)
    df_races = pd.concat([df_races, df_aux])
    

/carreras/jornada/20190303
Error en carrera 0    Domingo, 3 de Marzo de 2019
Name: 0, dtype: object 0    Hora: 12:30
Name: 1, dtype: object
Error en carrera 0    Domingo, 3 de Marzo de 2019
Name: 0, dtype: object 0    Hora: 13:30
Name: 1, dtype: object
Error en carrera 0    Domingo, 3 de Marzo de 2019
Name: 0, dtype: object 0    Hora: 14:00
Name: 1, dtype: object
Error en carrera 0    Domingo, 3 de Marzo de 2019
Name: 0, dtype: object 0    Hora: 14:30
Name: 1, dtype: object
/carreras/jornada/20190310
/carreras/jornada/20190317
/carreras/jornada/20190324
/carreras/jornada/20190331
/carreras/jornada/20190407
/carreras/jornada/20190414
/carreras/jornada/20190421
/carreras/jornada/20190428
/carreras/jornada/20190505
/carreras/jornada/20190512
/carreras/jornada/20190519
/carreras/jornada/20190526
/carreras/jornada/20190602
/carreras/jornada/20190609
/carreras/jornada/20190615
/carreras/jornada/20190622
/carreras/jornada/20190629
/carreras/jornada/20190704
/carreras/jornada/20190718
/carrera

Se obtiene el tamaño de la base de datos

In [88]:
df_races.shape

(5170, 16)

Se añade una columna que indica si el caballo es ganador o no

In [89]:
df_races['ganador'] = np.where(df_races['Puesto']== 1, 1, 0)

In [90]:
df_races[['Puesto', 'ganador']][15:30]

,Puesto,ganador
4,5,0
5,6,0
6,7,0
7,8,0
8,9,0
9,10,0
10,97,0
0,1,1
1,2,0
2,3,0


In [91]:
df_races.to_csv(r'raceDf.csv')
df_races.to_excel(r'raceDf.xlsx')

In [92]:
df_races

,Puesto,Caballo,Peso,Edad,Dist.,Mant.,Divid.,Propietario,Preparador,Jinete,Terreno,Distancia,Tipo,Tiempo,CodCarrera,CodDia,ganador
0,1,ARENAL (IRE),5800,3,NaN,1,350,FEROZ,P.OLAVE,J.L. BORREGO,H-Bueno,1400,Liso,85,44038,20190303,1
1,2,QUINCHIA (IRE),5650,3,CABEZA,9,1440,COCHETEUX,M.ALVAREZ,J.GELABERT,H-Bueno,1400,Liso,85,44038,20190303,0
2,3,TAZONES,5800,3,1 C 3/4,8,1380,YEGUADA ROCIO,G.ARIZKORRETA,V. JANACEK,H-Bueno,1400,Liso,85,44038,20190303,0
3,4,COSSIO,5800,3,2 C 1/4,3,230,INCITATUS RACING,M.AUGELLI,J.L. MARTINEZ,H-Bueno,1400,Liso,85,44038,20190303,0
4,5,RED HUT RED (IRE),5650,3,1 C 1/2,10,1170,DARIUSH KHAN,O.ANAYA,M.E.FERNANDEZ,H-Bueno,1400,Liso,85,44038,20190303,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,8,EVA GLITTERS (FR),5450,5,2 C,11,3710,L & M RACING,D.TEIXEIRA,M.E.FERNANDEZ,Arena,1700,Liso,104,46235,20201016,0
8,9,IMPRESSIONANT (USA),6350-6200,5,6 C 3/4,1,1050,ALISARES,J.A.RODRIGUEZ,I.MELGAREJO,Arena,1700,Liso,104,46235,20201016,0
9,10,WINDY YORK (IRE),6200,7,1 C 1/4,2,700,SHOSHOLOZA,M&M RACING,V. JANACEK,Arena,1700,Liso,104,46235,20201016,0
10,11,RADAMES(IRE),6150-5850,5,4 C 1/4,3,420,YEGUADA AGF,G.ARIZKORRETA,N.SACCU,Arena,1700,Liso,104,46235,20201016,0
